########## choose stock list (aka market) ##################

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date

In [2]:
## To acquire stock list in NASDAQ ###
ex = pd.read_csv('stock_info.csv')
df = pd.DataFrame(ex['Ticker'].astype('str'))
df = df.rename(columns={'Ticker' : 'ticker'})
df = df.reset_index(drop=True)
df

,ticker
0,A
1,AA
2,AAN
3,AAT
4,AAV
...,...
5777,ZN
5778,ZNGA
5779,ZOLT
5780,ZOOM


### Acquire informations from yfinance function >> .info

In [3]:
# 1st pulling date for this data #
df['date_pulling'] = date.today()

In [4]:
# 2nd acquiring information from .info #
info_attribute_list = [
    'industry',
    'sector',
    'marketCap',
    'currentPrice',
    'enterpriseValue',
    'totalCashPerShare'
]
# 3rd acquiring information from .quarterly_balance_sheet #
balancesheet_list = [
    'Total Assets',
    'Current Liabilities'
]

# 4th acquiring information from .quarterly_financials #
ttm = 4
financials_list = [
    'EBIT',
    'Operating Income'
]

info_attribute_list_buffer = []
balancesheet_list_buffer = []
financials_list_buffer = []
for i in range(len(df)):
    # print(i, " ",df['ticker'][i])
    yfticker = yf.Ticker(df['ticker'][i])

    # 2nd acquiring information from .info #
    info_attribute_list_buffer_r = []
    for j in range(len(info_attribute_list)):
        try:
            info_attribute_list_buffer_r.append(yfticker.info[info_attribute_list[j]])
        except:
            info_attribute_list_buffer_r.append(None)
    info_attribute_list_buffer.append(info_attribute_list_buffer_r)

    # 3rd acquiring information from .balancesheet #
    balancesheet_list_buffer_r = []
    for j in range(len(balancesheet_list)):
        try:
            balancesheet_list_buffer_r.append(yfticker.quarterly_balance_sheet.loc[yfticker.quarterly_balance_sheet.index == balancesheet_list[j]].values[0][0])
        except:
            balancesheet_list_buffer_r.append(None)
    balancesheet_list_buffer.append(balancesheet_list_buffer_r)

    # 4th acquiring information from .quarterly_financials #
    financials_list_buffer_r = []
    for j in range(len(financials_list)):
        try:
            financials_list_buffer_r.append(np.array([yfticker.quarterly_financials.loc[yfticker.quarterly_financials.index == financials_list[j]].values[0][i] for i in range(ttm)]).sum())
        except:
            financials_list_buffer_r.append(None)
    try:
        financials_list_buffer_r.append(yfticker.quarterly_financials.columns[0])
    except:
        financials_list_buffer_r.append(None)
    financials_list_buffer.append(financials_list_buffer_r)

financials_list.append('ttm_latest')
df = df.join(pd.DataFrame(info_attribute_list_buffer, columns=info_attribute_list))
df = df.join(pd.DataFrame(balancesheet_list_buffer, columns=balancesheet_list))
df = df.join(pd.DataFrame(financials_list_buffer, columns=financials_list))


##### Drop NAN #############
df = df.dropna()
##### Drop where marketcap and EBIT < 0#############
df = df.loc[df['enterpriseValue'] >0]
df = df.loc[df['EBIT'] >0]
df = df.loc[df['Operating Income'] >0]
df = df.reset_index(drop=True)

df.to_csv('data_stock_nyse.csv',index= False)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=oj8JRHy%2FYbf
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AEL?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AEL&crumb=oj8JRHy%2FYbf
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AFW?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AFW&crumb=oj8JRHy%2FYbf
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AGM.A?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yah

############################################################################

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date
df = pd.read_csv('data_stock_nyse.csv')
print('sector', df['sector'].unique())
print('industry', df['industry'].unique())

sector ['Healthcare' 'Real Estate' 'Industrials' 'Consumer Defensive'
 'Technology' 'Utilities' 'Basic Materials' 'Consumer Cyclical'
 'Financial Services' 'Communication Services' 'Energy']
industry ['Diagnostics & Research' 'REIT—Diversified' 'Drug Manufacturers—General'
 'Specialty Business Services' 'Medical Devices' 'Grocery Stores'
 'Engineering & Construction' 'Information Technology Services'
 'REIT—Retail' 'Farm Products' 'Security & Protection Services'
 'Utilities—Regulated Electric' 'Gold' 'Apparel Retail'
 'Rental & Leasing Services' 'Utilities—Diversified'
 'Farm & Heavy Construction Machinery' 'REIT—Hotel & Motel'
 'Textile Manufacturing' 'Aerospace & Defense' 'Industrial Distribution'
 'Insurance Brokers' 'Auto Parts' 'Semiconductors'
 'Specialty Industrial Machinery' 'Asset Management' 'REIT—Residential'
 'REIT—Specialty' 'Telecom Services' 'Auto & Truck Dealerships'
 'Oil & Gas E&P' 'Specialty Chemicals' 'Electronic Components'
 'REIT—Office' 'Electronics & Computer D

In [6]:

### Next I perform calculations for MF_ROC and MF_EY ###
represents_earning = 'Operating Income'
# represents_earning = 'EBIT'

df['MF_ROC'] = df[represents_earning]/(df['Total Assets'] - df['Current Liabilities'])
df['MF_EY'] = df[represents_earning]/df['enterpriseValue']
df

,ticker,date_pulling,industry,sector,marketCap,currentPrice,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY
0,A,2024-08-18,Diagnostics & Research,Healthcare,4.085821e+10,140.04,4.191319e+10,5.727,1.085600e+10,1.958000e+09,1.417000e+09,1.288000e+09,2024-04-30,0.144752,0.030730
1,AAT,2024-08-18,REIT—Diversified,Real Estate,1.956369e+09,25.38,3.088659e+09,2.026,2.992932e+09,1.156700e+08,1.351990e+08,1.218810e+08,2024-06-30,0.042360,0.039461
2,ABBV,2024-08-18,Drug Manufacturers—General,Healthcare,3.424933e+11,193.90,4.003180e+11,7.449,1.419370e+11,4.191500e+10,9.571000e+09,1.372100e+10,2024-06-30,0.137180,0.034275
3,ABM,2024-08-18,Specialty Business Services,Industrials,3.396138e+09,54.11,4.731239e+09,0.967,4.929500e+09,1.179100e+09,4.059000e+08,4.006000e+08,2024-04-30,0.106815,0.084671
4,ABT,2024-08-18,Medical Devices,Healthcare,1.931289e+11,111.00,2.009967e+11,4.149,7.301700e+10,1.376000e+10,7.115000e+09,6.482000e+09,2024-06-30,0.109388,0.032249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301,WWD,2024-08-18,Aerospace & Defense,Industrials,9.286552e+09,155.66,9.929068e+09,5.168,4.320309e+09,9.130920e+08,4.966980e+08,4.380080e+08,2024-06-30,0.128553,0.044114
1302,WYNN,2024-08-18,Resorts & Casinos,Consumer Cyclical,8.467580e+09,76.29,1.736902e+10,26.173,1.328980e+10,2.601026e+09,1.232325e+09,1.284892e+09,2024-06-30,0.120210,0.073976
1303,YY,2024-08-18,Internet Content & Information,Communication Services,2.085179e+09,35.15,3.920537e+10,2.653,8.556583e+09,3.121261e+09,3.148030e+08,3.600400e+07,2024-03-31,0.006624,0.000918
1304,ZBRA,2024-08-18,Communication Equipment,Technology,1.775088e+10,344.14,1.969487e+10,7.968,7.490000e+09,1.522000e+09,3.400000e+08,4.810000e+08,2024-06-30,0.080597,0.024423


In [7]:
### Drop Utilities, Energy, and Financial Services  as suggested from the book ###
sectortoexclude = [
    'Utilities',
    'Energy',
    'Financial Services',
    'Real Estate'
]
for i in sectortoexclude:
    print('exclude sector > ',i)
    try:
         
        df = df.loc[df['sector'] != i]
    except:
        None
        
industrytoexclude =[
    'Engineering & Construction',
    'Building Products & Equipment',
    'Building Materials'
]
for i in industrytoexclude:
    print('exclude industry > ',i)
    try:
         
        df = df.loc[df['industry'] != i]
    except:
        None

df = df.reset_index(drop=True)

exclude sector >  Utilities
exclude sector >  Energy
exclude sector >  Financial Services
exclude sector >  Real Estate
exclude industry >  Engineering & Construction
exclude industry >  Building Products & Equipment
exclude industry >  Building Materials


In [8]:
#### Select market size by choosing ' market ' ###
market =  50000000### in USD
df_market = df.loc[df['marketCap'] >= market]
df_market = df_market.reset_index(drop=True)

In [9]:
### Ranking regarding MFs####

df_market['Ranking_MF_ROC'] = df_market['MF_ROC'].rank()
df_market['Ranking_MF_EY'] = df_market['MF_EY'].rank()
df_market['Ranking_MF'] = df_market['Ranking_MF_ROC'] + df_market['Ranking_MF_EY']

df_market.loc[df_market['Ranking_MF'] == df_market['Ranking_MF'].max()]

,ticker,date_pulling,industry,sector,marketCap,currentPrice,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
345,NVO,2024-08-18,Biotechnology,Healthcare,6.017052e+11,134.89,5.957051e+11,14.055,3.693830e+11,1.879090e+11,1.166360e+11,1.114590e+11,2024-06-30,0.614187,0.187104,945.0,923.0,1868.0


In [10]:
numstocks = 50
df_sorted = df_market.sort_values(by=['Ranking_MF'],ascending=False)
df_sorted = df_sorted.reset_index(drop=True)
df_sorted[:numstocks]

,ticker,date_pulling,industry,sector,marketCap,currentPrice,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
0,NVO,2024-08-18,Biotechnology,Healthcare,6.017052e+11,134.890,5.957051e+11,14.055,3.693830e+11,1.879090e+11,1.166360e+11,1.114590e+11,2024-06-30,0.614187,0.187104,945.0,923.0,1868.0
1,SIGA,2024-08-18,Drug Manufacturers—Specialty & Generic,Healthcare,7.486639e+08,10.490,6.433619e+08,1.499,1.926586e+08,2.137805e+07,1.027231e+08,1.027231e+08,2024-06-30,0.599736,0.159666,943.0,906.0,1849.0
2,OMAB,2024-08-18,Airports & Air Services,Industrials,3.190325e+09,66.190,3.498960e+10,4.169,2.590749e+10,4.752860e+09,7.834555e+09,7.527290e+09,2024-06-30,0.355822,0.215129,916.0,931.0,1847.0
3,RCMT,2024-08-18,Conglomerates,Industrials,1.454844e+08,19.070,1.716414e+08,0.456,1.182780e+08,5.764900e+07,2.389200e+07,2.389200e+07,2024-06-30,0.394069,0.139197,927.0,893.0,1820.0
4,HY,2024-08-18,Farm & Heavy Construction Machinery,Industrials,1.046566e+09,59.800,1.491881e+09,4.145,2.115900e+09,1.171600e+09,2.976000e+08,2.867000e+08,2024-06-30,0.303611,0.192173,890.0,924.0,1814.0
5,DDS,2024-08-18,Department Stores,Consumer Cyclical,5.734392e+09,353.240,5.212588e+09,66.074,3.937156e+09,1.130290e+09,9.238860e+08,8.957790e+08,2024-04-30,0.319138,0.171849,898.0,912.0,1810.0
6,TZOO,2024-08-18,Advertising Agencies,Communication Services,1.458847e+08,11.870,1.467276e+08,1.023,5.151300e+07,3.282700e+07,1.719000e+07,1.719000e+07,2024-06-30,0.919940,0.117156,950.0,853.0,1803.0
7,PSIX,2024-08-18,Specialty Industrial Machinery,Industrials,3.968264e+08,17.260,5.303873e+08,1.253,3.075620e+08,2.438430e+08,6.045200e+07,6.045200e+07,2024-06-30,0.948728,0.113977,951.0,845.0,1796.0
8,MSB,2024-08-18,Steel,Basic Materials,2.204160e+08,16.800,2.044278e+08,1.649,2.513116e+07,4.477027e+06,2.284348e+07,2.284348e+07,2024-04-30,1.106001,0.111744,952.0,841.0,1793.0
9,BKE,2024-08-18,Apparel Retail,Consumer Cyclical,2.178905e+09,42.910,2.172612e+09,5.827,9.060700e+08,2.052800e+08,2.597250e+08,2.597250e+08,2024-04-30,0.370617,0.119545,921.0,861.0,1782.0


In [11]:
df_sorted_2 = df_sorted[:numstocks].copy()
df_sorted_2 = df_sorted_2.sort_values(by=['totalCashPerShare'],ascending=False)
df_sorted_2 = df_sorted_2.reset_index(drop=True)
df_sorted_2

,ticker,date_pulling,industry,sector,marketCap,currentPrice,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
0,NVR,2024-08-18,Residential Construction,Consumer Cyclical,2.667941e+10,8668.930,2.528608e+10,792.331,6.407288e+09,1.260029e+09,2.055058e+09,2.050196e+09,2024-06-30,0.398308,0.081080,928.0,704.0,1632.0
1,TLK,2024-08-18,Telecom Services,Communication Services,1.870294e+10,18.880,6.996729e+13,283.892,2.859920e+14,8.317300e+13,4.434200e+13,4.457200e+13,2024-06-30,0.219762,0.637040,818.0,949.0,1767.0
2,MOH,2024-08-18,Healthcare Plans,Healthcare,2.050824e+10,349.970,1.419124e+10,148.481,1.518500e+10,7.836000e+09,1.535000e+09,1.535000e+09,2024-06-30,0.208872,0.108165,802.0,830.0,1632.0
3,TSM,2024-08-18,Semiconductors,Technology,9.051783e+11,174.540,3.534018e+12,79.010,5.787891e+12,1.026180e+12,1.082195e+12,1.023834e+12,2024-06-30,0.215014,0.289708,812.0,940.0,1752.0
4,UTHR,2024-08-18,Biotechnology,Healthcare,1.428123e+10,320.990,1.180972e+10,66.789,6.723200e+09,8.323000e+08,1.456700e+09,1.263300e+09,2024-06-30,0.214449,0.106971,811.0,826.0,1637.0
5,DDS,2024-08-18,Department Stores,Consumer Cyclical,5.734392e+09,353.240,5.212588e+09,66.074,3.937156e+09,1.130290e+09,9.238860e+08,8.957790e+08,2024-04-30,0.319138,0.171849,898.0,912.0,1810.0
6,ASR,2024-08-18,Airports & Air Services,Industrials,8.346212e+09,282.460,8.908710e+10,49.990,7.299319e+10,4.898742e+09,1.674405e+10,1.580910e+10,2024-03-31,0.232164,0.177457,832.0,917.0,1749.0
7,NTES,2024-08-18,Electronic Gaming & Multimedia,Communication Services,5.794646e+10,89.910,1.827993e+11,41.908,1.950649e+11,6.004937e+10,2.811154e+10,2.811154e+10,2024-03-31,0.208210,0.153784,800.0,902.0,1702.0
8,POWL,2024-08-18,Electrical Equipment & Parts,Industrials,2.020304e+09,168.530,1.647174e+09,31.202,8.689830e+08,4.184350e+08,1.524610e+08,1.524610e+08,2024-06-30,0.338390,0.092559,906.0,770.0,1676.0
9,IDCC,2024-08-18,Software—Application,Technology,3.384733e+09,134.690,3.113497e+09,30.257,1.636295e+09,6.993470e+08,3.677850e+08,3.166220e+08,2024-06-30,0.337929,0.101693,905.0,804.0,1709.0
